In [2]:
import torch
import torch_geometric
from torch_geometric.data import Data, DataLoader
import deepchem as dc
from rdkit import Chem
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data, InMemoryDataset

import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from torch.optim.lr_scheduler import ReduceLROnPlateau

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\deepchem\models\torch_models\__init__.py)


In [3]:
from torch_geometric.data import InMemoryDataset, Data

class DeepChemToPyGDataset(InMemoryDataset):
    def __init__(self, data_list, transform=None):
        """
        Converts a list of PyG `Data` objects into an InMemoryDataset.
        
        Args:
            data_list (list of Data): List of PyG Data objects.
            transform (callable, optional): Optional transform applied to each data object.
        """
        super(DeepChemToPyGDataset, self).__init__('.', transform, None, None)
        self.data, self.slices = self.collate(data_list)

    def get(self, idx):
        """
        Retrieves the data object at the specified index.
        """
        return super().get(idx)


In [4]:
pyg_dataset = torch.load('../../data/curated dataset/cleaned_frag_pyg_dataset.pth')

C:\Users\sarab\AppData\Local\Temp\ipykernel_19664\3827785572.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pyg_dataset = torch.load('../../data/curated dataset/cleaned

In [4]:
node_features = pyg_dataset[0].x.shape[1]  # Should be 134 based on your description
edge_features = pyg_dataset[0].edge_attr.shape[1]  # Should be 6 based on your description


In [5]:
#Defining the generative model here it is GAE model
class Autoencoder(nn.Module):
    def __init__(self, node_features, edge_features, hidden_dim, latent_dim):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder(node_features, edge_features, hidden_dim, latent_dim)
        self.decoder = Decoder(latent_dim, hidden_dim, node_features, edge_features)

    def encode(self, x, edge_index, edge_attr):
        return self.encoder(x, edge_index, edge_attr)

    def decode(self, z, edge_index):
        return self.decoder(z, edge_index)

    def forward(self, x, edge_index, edge_attr):
        z, _ = self.encode(x, edge_index, edge_attr)  # Ignore logvar
        reconstructed_x, reconstructed_edge_attr = self.decode(z, edge_index)
        return reconstructed_x, reconstructed_edge_attr

    def generate_multiple_outputs(self, x, edge_index, edge_attr, num_samples=5):
        outputs = []
        with torch.no_grad():
            z, _ = self.encode(x, edge_index, edge_attr)
            for _ in range(num_samples):
                # Add noise to z (sampling from normal distribution)
                z_noisy = z + torch.randn_like(z) * 0.5  # Adjust std deviation for noise level
                reconstructed_x, reconstructed_edge_attr = self.decode(z_noisy, edge_index)
                outputs.append((reconstructed_x, reconstructed_edge_attr , edge_index))
        return outputs

    def generate_molecule(self, num_nodes):
        self.eval()
        with torch.no_grad():
            # Sample from the latent space
            z = torch.randn(num_nodes, latent_dim)

            # Create a fully connected edge_index
            edge_index = torch.combinations(torch.arange(num_nodes), 2).t()

            # Decode
            reconstructed_x, reconstructed_edge_attr = self.decode(z, edge_index)

            return reconstructed_x, edge_index, reconstructed_edge_attr

class Encoder(nn.Module):
    def __init__(self, node_features, edge_features, hidden_dim, latent_dim):
        super(Encoder, self).__init__()
        self.conv1 = SAGEConv(node_features + edge_features, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim)
        self.conv3 = SAGEConv(hidden_dim, hidden_dim)  # Additional layer
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)

    def forward(self, x, edge_index, edge_attr):
        edge_attr_expanded = torch.zeros((x.size(0), edge_features)).to(x.device)
        edge_attr_expanded[edge_index[0]] = edge_attr
        x = torch.cat([x, edge_attr_expanded], dim=1)

        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))  # Added layer
        return self.fc_mu(x), None  # Return mu and None for logvar

class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, node_features, edge_features):
        super(Decoder, self).__init__()
        self.fc = nn.Linear(latent_dim, hidden_dim)
        self.node_decoder = nn.Linear(hidden_dim, node_features)
        self.edge_decoder = nn.Linear(hidden_dim * 2, edge_features)

    def forward(self, z, edge_index):
        h = F.relu(self.fc(z))
        reconstructed_x = self.node_decoder(h)

        # Edge reconstruction
        row, col = edge_index
        edge_h = torch.cat([h[row], h[col]], dim=1)
        reconstructed_edge_attr = self.edge_decoder(edge_h)

        return reconstructed_x, reconstructed_edge_attr

In [6]:
#Defining the reconstruction loss. It is MSE for GAE
def loss_function(recon_x, x, recon_edge_attr, edge_attr):
    # Reconstruction losses
    node_recon_loss = F.mse_loss(recon_x, x)
    edge_recon_loss = F.mse_loss(recon_edge_attr, edge_attr)

    return node_recon_loss + edge_recon_loss

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs=60

In [8]:
import optuna
import torch
import torch.nn.functional as F
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
from torch_geometric.loader import DataLoader as PyGDataLoader
from multiprocessing import cpu_count

# Define objective function for hyperparameter tuning
def objective(trial):
    # Define hyperparameters to search
    hidden_dim = trial.suggest_int("hidden_dim",64,512, step=64)
    latent_dim = trial.suggest_int("latent_dim", 32, 128, step=32)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8,16, 32, 64])
    patience = trial.suggest_int("patience", 3, 10)
    # Print the current trial's hyperparameters
    print(f"\n[Trial {trial.number}] Starting with parameters: {trial.params}")
    # Prepare data loaders
    train_ratio = 0.8
    valid_ratio = 0.2

    # Calculate the number of training and validation samples
    num_train = int(len(pyg_dataset) * train_ratio)
    num_valid = len(pyg_dataset) - num_train

    # Split the dataset into training and validation sets
    train_dataset, valid_dataset = random_split(pyg_dataset, [num_train, num_valid])

    # Create DataLoader for training dataset
    train_loader = PyGDataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Create DataLoader for validation dataset
    val_loader = PyGDataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    # Initialize model
    model = Autoencoder(node_features, edge_features, hidden_dim, latent_dim).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=patience // 2)

    # Early stopping parameters
    best_val_loss = float('inf')
    epochs_without_improvement = 0

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for batch in train_loader:
            optimizer.zero_grad()
            x = batch.x.to(device)
            edge_index = batch.edge_index.to(device)
            edge_attr = batch.edge_attr.to(device)

            recon_x, recon_edge_attr = model(x, edge_index, edge_attr)
            loss = loss_function(recon_x, x, recon_edge_attr, edge_attr)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        #print(f"[Trial {trial.number}] Epoch {epoch+1}/{epochs}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for val_batch in val_loader:
                x_val = val_batch.x.to(device)
                edge_index_val = val_batch.edge_index.to(device)
                edge_attr_val = val_batch.edge_attr.to(device)

                recon_x_val, recon_edge_attr_val = model(x_val, edge_index_val, edge_attr_val)
                val_loss += loss_function(recon_x_val, x_val, recon_edge_attr_val, edge_attr_val).item()

        avg_val_loss = val_loss / len(val_loader)
        # Print training and validation losses
        print(f"[Trial {trial.number}] Epoch {epoch+1}/{epochs}, Training Loss: {avg_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")
        scheduler.step(avg_val_loss)

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= patience:
            break

    return best_val_loss


# Use multiprocessing to parallelize trials
# Run the study
if __name__ == "__main__":
    num_jobs = min(cpu_count(), 7)  # Use up to 4 cores or fewer if limited
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=200, n_jobs=num_jobs)

    # Output the best parameters along with the trial number
    best_trial = study.best_trial
    print(f"\nBest Trial Number: {best_trial.number}")
    print("Best Parameters:", best_trial.params)
    print("Best Value (Objective):", best_trial.value)



c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-12-05 22:59:36,935] A new study created in memory with name: no-name-6dfff094-30fd-40a0-b860-a8811e7cc8ab



[Trial 0] Starting with parameters: {'hidden_dim': 512, 'latent_dim': 96, 'learning_rate': 0.0010090916594610252, 'batch_size': 32, 'patience': 9}
[Trial 1] Starting with parameters: {'hidden_dim': 256, 'latent_dim': 96, 'learning_rate': 0.04375368047438927, 'batch_size': 16, 'patience': 4}


[Trial 2] Starting with parameters: {'hidden_dim': 64, 'latent_dim': 32, 'learning_rate': 1.4444360533043452e-05, 'batch_size': 64, 'patience': 10}

[Trial 3] Starting with parameters: {'hidden_dim': 320, 'latent_dim': 32, 'learning_rate': 2.7519810029384675e-05, 'batch_size': 8, 'patience': 4}

[Trial 4] Starting with parameters: {'hidden_dim': 128, 'latent_dim': 96, 'learning_rate': 0.04514113889129663, 'batch_size': 64, 'patience': 4}

[Trial 5] Starting with parameters: {'hidden_dim': 320, 'latent_dim': 128, 'learning_rate': 1.9731048129123216e-05, 'batch_size': 16, 'patience': 9}

[Trial 6] Starting with parameters: {'hidden_dim': 256, 'latent_dim': 128, 'learning_rate': 0.062105151767771906

[I 2024-12-05 23:00:29,634] Trial 4 finished with value: 0.1250482236345609 and parameters: {'hidden_dim': 128, 'latent_dim': 96, 'learning_rate': 0.04514113889129663, 'batch_size': 64, 'patience': 4}. Best is trial 4 with value: 0.1250482236345609.


[Trial 5] Epoch 2/60, Training Loss: 0.0556, Validation Loss: 0.0437
[Trial 4] Epoch 6/60, Training Loss: 0.1255, Validation Loss: 0.1251

[Trial 7] Starting with parameters: {'hidden_dim': 192, 'latent_dim': 96, 'learning_rate': 0.04111250258244466, 'batch_size': 8, 'patience': 3}
[Trial 1] Epoch 2/60, Training Loss: 0.1270, Validation Loss: 0.1259
[Trial 2] Epoch 7/60, Training Loss: 0.1945, Validation Loss: 0.1825
[Trial 0] Epoch 3/60, Training Loss: 0.0044, Validation Loss: 0.0037
[Trial 2] Epoch 8/60, Training Loss: 0.1728, Validation Loss: 0.1612
[Trial 2] Epoch 9/60, Training Loss: 0.1530, Validation Loss: 0.1435
[Trial 0] Epoch 4/60, Training Loss: 0.0036, Validation Loss: 0.0032
[Trial 2] Epoch 10/60, Training Loss: 0.1380, Validation Loss: 0.1311
[Trial 6] Epoch 3/60, Training Loss: 0.3286, Validation Loss: 0.2163
[Trial 5] Epoch 3/60, Training Loss: 0.0344, Validation Loss: 0.0281
[Trial 1] Epoch 3/60, Training Loss: 0.1269, Validation Loss: 0.1274
[Trial 2] Epoch 11/60, Tra

[I 2024-12-05 23:03:04,613] Trial 7 finished with value: 0.12220564434925715 and parameters: {'hidden_dim': 192, 'latent_dim': 96, 'learning_rate': 0.04111250258244466, 'batch_size': 8, 'patience': 3}. Best is trial 7 with value: 0.12220564434925715.


...

[Trial 197] Epoch 41/60, Training Loss: 0.0016, Validation Loss: 0.0019
[Trial 196] Epoch 50/60, Training Loss: 0.0016, Validation Loss: 0.0019
[Trial 198] Epoch 34/60, Training Loss: 0.0017, Validation Loss: 0.0022
[Trial 198] Epoch 35/60, Training Loss: 0.0016, Validation Loss: 0.0022
[Trial 196] Epoch 51/60, Training Loss: 0.0016, Validation Loss: 0.0019
[Trial 198] Epoch 36/60, Training Loss: 0.0016, Validation Loss: 0.0022
[Trial 196] Epoch 52/60, Training Loss: 0.0016, Validation Loss: 0.0019
[Trial 198] Epoch 37/60, Training Loss: 0.0016, Validation Loss: 0.0022
[Trial 196] Epoch 53/60, Training Loss: 0.0016, Validation Loss: 0.0019
[Trial 198] Epoch 38/60, Training Loss: 0.0016, Validation Loss: 0.0022
[Trial 196] Epoch 54/60, Training Loss: 0.0016, Validation Loss: 0.0019
[Trial 198] Epoch 39/60, Training Loss: 0.0016, Validation Loss: 0.0022
[Trial 196] Epoch 55/60, Training Loss: 0.0016, Validation Loss: 0.0019
[Trial 198] Epoch 40/60, Training Loss: 0.0016, Validation Loss:

[I 2024-12-06 08:43:58,851] Trial 196 finished with value: 0.0019244492422634115 and parameters: {'hidden_dim': 320, 'latent_dim': 96, 'learning_rate': 0.004270859625364796, 'batch_size': 16, 'patience': 4}. Best is trial 130 with value: 0.0015067179968658213.


[Trial 196] Epoch 60/60, Training Loss: 0.0015, Validation Loss: 0.0019
[Trial 198] Epoch 45/60, Training Loss: 0.0016, Validation Loss: 0.0022
[Trial 198] Epoch 46/60, Training Loss: 0.0016, Validation Loss: 0.0022
[Trial 198] Epoch 47/60, Training Loss: 0.0016, Validation Loss: 0.0022
[Trial 198] Epoch 48/60, Training Loss: 0.0016, Validation Loss: 0.0022
[Trial 198] Epoch 49/60, Training Loss: 0.0016, Validation Loss: 0.0022
[Trial 198] Epoch 50/60, Training Loss: 0.0016, Validation Loss: 0.0022
[Trial 198] Epoch 51/60, Training Loss: 0.0016, Validation Loss: 0.0022
[Trial 198] Epoch 52/60, Training Loss: 0.0016, Validation Loss: 0.0022
[Trial 198] Epoch 53/60, Training Loss: 0.0016, Validation Loss: 0.0022
[Trial 198] Epoch 54/60, Training Loss: 0.0016, Validation Loss: 0.0022
[Trial 198] Epoch 55/60, Training Loss: 0.0016, Validation Loss: 0.0022
[Trial 198] Epoch 56/60, Training Loss: 0.0016, Validation Loss: 0.0022
[Trial 198] Epoch 57/60, Training Loss: 0.0016, Validation Loss:

[I 2024-12-06 08:44:50,237] Trial 198 finished with value: 0.0021574872138444335 and parameters: {'hidden_dim': 320, 'latent_dim': 128, 'learning_rate': 0.0008195940486637011, 'batch_size': 16, 'patience': 4}. Best is trial 130 with value: 0.0015067179968658213.


[Trial 198] Epoch 60/60, Training Loss: 0.0016, Validation Loss: 0.0022

Best Trial Number: 130
Best Parameters: {'hidden_dim': 320, 'latent_dim': 128, 'learning_rate': 0.001281030785701166, 'batch_size': 16, 'patience': 4}
Best Value (Objective): 0.0015067179968658213
